# Exercices SQL

Récupérer les données dans 'produits csv'

Les insérer dans une nouvelle base de deonnées qu’on aura conçue
Faire ça en python



In [109]:
import numpy as np
import pandas as pd

prod_df = pd.read_csv('./Produits - Feuille 1.csv')

print(prod_df.shape[0])

fam_df = prod_df.groupby(['FAMILLE ARTICLE']).count().drop(['LIBELLE ARTICLE', 'CONDITION', 'PU HT'], axis=1)
cond_df = prod_df.groupby(['CONDITION']).count().drop(['LIBELLE ARTICLE', 'FAMILLE ARTICLE', 'PU HT'], axis=1)

1863
           Code article
CONDITION              
BOITE               395
CARTO               188
SACHE               162
SAC                 131
BOUTE               112
PAQUE               110
KG                   99
POT                  79
FARDE                71
SEAU                 57
BIDON                54
PIECE                40
BARQU                40
PLATE                39
UNI                  39
FLACO                38
SQUEE                36
LOT                  29
ETUI                 29
PACK                 26
BOCAL                26
BRICK                18
TUBO                  8
POCHE                 7
CAISS                 7
SPRAY                 7
TUBE                  4
BAC                   4
PINGO                 3
BIB                   3
COLIS                 1
BRK                   1


Une fois qu’on a ces infos, on sait qu’on devra créer une table pour chacunes de ces informations en y associant un identifiant

Se connecter avec PyMysql

In [7]:
import mysql.connector
import random

db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'sakila'
)

cur = db.cursor()

cur.execute("SELECT COUNT(*) FROM actor")
res = cur.fetchall()
print(res)


[(200,)]


Créons la Base de Données sur mon server

In [8]:
#create random id
actor_ids = list(range(1,201))
random.shuffle(actor_ids)
print(actor_ids[0])

sql = '''
SELECT f.title, a.first_name, a.last_name, a.actor_id
FROM film_actor AS fa
JOIN film AS f
ON fa.film_id = f.film_id
JOIN actor AS a
ON fa.actor_id = a.actor_id
WHERE fa.actor_id = %s;
'''

cur.execute(sql, (actor_ids[0],))
res = cur.fetchall()
print(len(res))


34
25


Essayons de créer une table sur ma base perso «produ1»

In [214]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'produ1'
)

cur = db.cursor()

cur.execute("DROP TABLE produit")
cur.execute("DROP TABLE famille")
cur.execute("DROP TABLE condit")

req = ['''
CREATE TABLE IF NOT EXISTS produit (
    prod_id INT NOT NULL AUTO_INCREMENT,
    fam_id INT,
    cond_id INT,
    code INT,
    libelle VARCHAR(80) NOT NULL UNIQUE,
    prix DECIMAL(6,2) NOT NULL,
    PRIMARY KEY (prod_id)
)''','''
CREATE TABLE IF NOT EXISTS famille (
    fam_id INT,
    nom VARCHAR(80) NOT NULL UNIQUE,
    PRIMARY KEY (fam_id)
)''','''
CREATE TABLE IF NOT EXISTS condit (
    cond_id INT,
    nom VARCHAR(80) NOT NULL UNIQUE,
    PRIMARY KEY (cond_id)
)
''']

for r in req : cur.execute(r)

cur.execute("SHOW TABLES")

for x in cur: print(x)

('condit',)
('famille',)
('produit',)


In [215]:

# INSERTS INTO FAMILLE
# purge de la table
cur.execute('TRUNCATE TABLE famille')

# préparer les données
fam_df = fam_df.sort_values('Code article', ascending=False)
famindex = fam_df.reset_index()
famindex['Code article'] = range(1,31)
famindex.columns = ('nom', 'fam_id')
print(famindex)

# création d’une table des inserts
inserts = []
for i, r in famindex.iterrows():
    inserts.append(f"({r['fam_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO famille (fam_id, nom)
VALUES
{', '.join(inserts)}
'''

# cur.execute(sql)
# db.commit()



                                    nom  fam_id
0                              SURGELES       1
1                  SAUCES ET CONDIMENTS       2
2                               JETABLE       3
3                     BOISSONS DIVERSES       4
4   CONSERVES LEGUMES ET PLATS PREPARES       5
5                              FROMAGES       6
6                    EPICES ET AROMATES       7
7             PATES ET AUTRES FECULENTS       8
8               PRODUITS DE  PATISSERIE       9
9         BISCOTTES ET DERIVES BISCUITS      10
10        PRODUITS HYGIENE ET ENTRETIEN      11
11                          CHARCUTERIE      12
12                     CONFITURE - MIEL      13
13                   PRODUITS DE LA MER      14
14                  CONSERVES DE FRUITS      15
15                               HUILES      16
16                             CREMERIE      17
17          FONDS ET SAUCES DESHYDRATES      18
18                          FRUITS SECS      19
19                                SUCRE 

In [216]:
# INSERTS INTO CONDITIONNEMENTS
# purge de la table
cur.execute('TRUNCATE TABLE condit')

# préparer les données
cond_df = cond_df.sort_values('Code article', ascending=False)
condindex = cond_df.reset_index()
condindex['Code article'] = range(1,condindex.shape[0]+1)
condindex.columns = ('nom', 'cond_id')

# création d’une table des inserts
inserts = []
for i, r in condindex.iterrows():
    inserts.append(f"({r['cond_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO condit (cond_id, nom)
VALUES
{', '.join(inserts)}
'''

cur.execute(sql)
db.commit()

In [217]:
# remplir la table principale

cur.execute('TRUNCATE TABLE produit')

# création de dictionnaires pour remplacer les noms de familles et conditions par leurs identifiants respectifs
fam_dic = famindex.to_dict('tight')
cond_dic = condindex.to_dict('tight')
repl_dict = {
    "FAMILLE ARTICLE": dict(fam_dic['data']),
    "CONDITION": dict(cond_dic['data'])
}
#boucle de remplacement
main_df = prod_df.copy()
for k,v in repl_dict.items() : {
    main_df[k].replace(v, inplace = True)
}
    
main_df.columns = ['code', 'libelle', 'fam_id', 'cond_id', 'prix']

inserts = []
for i, r in main_df.iterrows():
    tup = []
    tup.append(0)
    tup.append(r['fam_id'])
    tup.append(r['cond_id'])
    tup.append(r['code'])
    tup.append(r['libelle'])
    tup.append(float(r['prix'].replace(",",".")))
    tup = tuple(tup)
    inserts.append(tup)

sql = '''
INSERT INTO produit (prod_id, fam_id, cond_id, code, libelle, prix)
VALUES (%s, %s, %s, %s, %s, %s)
'''

cur.executemany(sql, inserts)
db.commit()


[(0, 16, 5, 1054, 'HUILE OLIVE TRUFFE 250ML', 12.15), (0, 16, 5, 1055, 'HUILE TRUFFE BLANCHE 250 ML EMPORIO', 12.0), (0, 16, 11, 1070, 'HUILE PEPIN DE RAISIN 5L', 17.97), (0, 16, 5, 1072, 'HUILE PEPIN DE RAISIN 1L', 3.91), (0, 16, 26, 1080, 'BOMBE A GRAISSE 600 ML', 4.93)]


In [219]:
# editer les tables pour y spécifier l’existance de clef étrangères

# sql = '''
# ALTER TABLE produit
# ADD CONSTRAINT FOREIGN KEY (fam_id)
# REFERENCES famille(fam_id)
# ON UPDATE CASCADE
# ON DELETE CASCADE;
# '''

sql = '''
ALTER TABLE produit
ADD FOREIGN KEY (fam_id)
REFERENCES famille(fam_id);
'''

cur.execute(sql)



IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`produ1`.`#sql-28c0_5f`, CONSTRAINT `produit_ibfk_1` FOREIGN KEY (`fam_id`) REFERENCES `famille` (`fam_id`))